In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
plt.rcParams['figure.facecolor'] = 'white'
import matplotlib.colors as colors
import cmocean.cm as cmo
from glob import glob
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

In [2]:
def prepro(ds):
    return ds.isel(y=slice(800, None))

In [3]:
grid_files = ["/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mask.nc", 
              "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mesh_hgr.nc",
              "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mesh_zgr.nc"]

In [4]:
grid = xr.open_mfdataset(grid_files, parallel=True, preprocess=prepro)

### Compute 5d work

In [ ]:
def calc_work(stress, ocean, uvt="u"):
    rho0 = 1025.
    work = stress * ocean
    if uvt == "t":
        work_t = work.data
    elif uvt == "u":
        work_t = np.ones(np.shape(work)) * np.nan
        work_t[:, :, 0:-1] = (work.data[:, :, 0:-1] 
                              + work.data[:, :, 1::]) / 2
    elif uvt == "v":
        work_t = np.ones(np.shape(work)) * np.nan
        work_t[:, 0:-1, :] = (work.data[:, 0:-1, :] 
                              + work.data[:, 1::, :]) / 2
    return (work_t / rho0)

In [ ]:
for y in np.arange(1996, 2016):
    print(y)
    gridU_data_filesREF = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" + str(y) + "/"
                                     + "CREG12.L75-REF08_*.5d_gridU.nc"))
    gridV_data_filesREF = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" + str(y) + "/"
                                     + "CREG12.L75-REF08_*.5d_gridV.nc"))
    gridT_data_filesREF = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" + str(y) + "/"
                                     + "CREG12.L75-REF08_*.5d_gridT.nc"))
    icemod_data_filesREF = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" + str(y) + "/"
                                     + "CREG12.L75-REF08_*.5d_icemod.nc"))
    UREF = xr.open_mfdataset(gridU_data_filesREF, parallel=True, preprocess=prepro)
    VREF = xr.open_mfdataset(gridV_data_filesREF, parallel=True, preprocess=prepro)
    TREF = xr.open_mfdataset(gridT_data_filesREF, parallel=True, preprocess=prepro)
    IREF = xr.open_mfdataset(icemod_data_filesREF, parallel=True, preprocess=prepro)
    lent = len(TREF.time_counter)
    if lent == 72:
        t = xr.cftime_range(start=TREF.time_counter.data[0], periods=73, freq="5D", calendar="noleap")
        time = xr.DataArray(t, coords={"time_counter": t})
        time.attrs = TREF.time_counter.attrs
    else:
        time = TREF.time_counter
    ds_REF_T = xr.Dataset(coords={"time_counter": time, 
                                  "nav_lat": TREF.nav_lat,"nav_lon": TREF.nav_lon})
    if lent == 72:
        dummy = (("time_counter", "y", "x"), np.zeros((len(time), len(TREF.y), len(TREF.x))))
        ds_REF_T["sozotwot"] = dummy
        ds_REF_T["sozoiwot"] = dummy
        ds_REF_T["sozoowot"] = dummy
        ds_REF_T["sometwot"] = dummy
        ds_REF_T["someiwot"] = dummy
        ds_REF_T["someowot"] = dummy
        ds_REF_T["sozotwot"][0:lent, :, :] = calc_work(UREF.sozotaux, 
                                                 UREF.vozocrtx.isel(depthu=0), uvt="u")
        ds_REF_T["sozoiwot"][0:lent, :, :] = calc_work(UREF.utau_iceoce * IREF.siconc, 
                                                 UREF.vozocrtx.isel(depthu=0), uvt="u")
        ds_REF_T["sozoowot"][0:lent, :, :] = calc_work(UREF.utau_atmoce * (1 - IREF.siconc), 
                                                 UREF.vozocrtx.isel(depthu=0), uvt="u")
        ds_REF_T["sometwot"][0:lent, :, :] = calc_work(VREF.sometauy, 
                                                 VREF.vomecrty.isel(depthv=0), uvt="v")
        ds_REF_T["someiwot"][0:lent, :, :] = calc_work(VREF.vtau_iceoce * IREF.siconc, 
                                                 VREF.vomecrty.isel(depthv=0), uvt="v")
        ds_REF_T["someowot"][0:lent, :, :] = calc_work(VREF.vtau_atmoce * (1 - IREF.siconc), 
                                                 VREF.vomecrty.isel(depthv=0), uvt="v")
        ds_REF_T["sozotwot"][lent, :, :] = ds_REF_T["sozotwot"][lent-1, :, :].data
        ds_REF_T["sozoiwot"][lent, :, :] = ds_REF_T["sozoiwot"][lent-1, :, :].data
        ds_REF_T["sozoowot"][lent, :, :] = ds_REF_T["sozoowot"][lent-1, :, :].data
        ds_REF_T["sometwot"][lent, :, :] = ds_REF_T["sometwot"][lent-1, :, :].data
        ds_REF_T["someiwot"][lent, :, :] = ds_REF_T["someiwot"][lent-1, :, :].data
        ds_REF_T["someowot"][lent, :, :] = ds_REF_T["someowot"][lent-1, :, :].data
    else:    
        ds_REF_T["sozotwot"] = (("time_counter", "y", "x"), calc_work(UREF.sozotaux, 
                                                              UREF.vozocrtx.isel(depthu=0), uvt="u"))
        ds_REF_T["sozoiwot"] = (("time_counter", "y", "x"), calc_work(UREF.utau_iceoce * IREF.siconc, 
                                                              UREF.vozocrtx.isel(depthu=0), uvt="u"))
        ds_REF_T["sozoowot"] = (("time_counter", "y", "x"), calc_work(UREF.utau_atmoce * (1 - IREF.siconc), 
                                                              UREF.vozocrtx.isel(depthu=0), uvt="u"))
        ds_REF_T["sometwot"] = (("time_counter", "y", "x"), calc_work(VREF.sometauy, 
                                                              VREF.vomecrty.isel(depthv=0), uvt="v"))
        ds_REF_T["someiwot"] = (("time_counter", "y", "x"), calc_work(VREF.vtau_iceoce * IREF.siconc, 
                                                              VREF.vomecrty.isel(depthv=0), uvt="v"))
        ds_REF_T["someowot"] = (("time_counter", "y", "x"), calc_work(VREF.vtau_atmoce * (1 - IREF.siconc), 
                                                              VREF.vomecrty.isel(depthv=0), uvt="v"))       
    ds_REF_T.to_netcdf("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-windwork/"
                       + "CREG12.L75-REF08_y" + str(y) + ".5d_windwork.nc")

In [ ]:
for y in np.arange(2051, 2071):
    print(y)
    gridU_data_filesFUT = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" + str(y) + "/"
                                     + "CREG12.L75-FUT08_*.5d_gridU.nc"))
    gridV_data_filesFUT = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" + str(y) + "/"
                                     + "CREG12.L75-FUT08_*.5d_gridV.nc"))
    gridT_data_filesFUT = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" + str(y) + "/"
                                     + "CREG12.L75-FUT08_*.5d_gridT.nc"))
    icemod_data_filesFUT = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" + str(y) + "/"
                                     + "CREG12.L75-FUT08_*.5d_icemod.nc"))
    UFUT = xr.open_mfdataset(gridU_data_filesFUT, parallel=True, preprocess=prepro)
    VFUT = xr.open_mfdataset(gridV_data_filesFUT, parallel=True, preprocess=prepro)
    TFUT = xr.open_mfdataset(gridT_data_filesFUT, parallel=True, preprocess=prepro)
    IFUT = xr.open_mfdataset(icemod_data_filesFUT, parallel=True, preprocess=prepro)
    lent = len(TFUT.time_counter)
    if lent == 72:
        t = xr.cftime_range(start=TFUT.time_counter.data[0], periods=73, freq="5D", calendar="noleap")
        time = xr.DataArray(t, coords={"time_counter": t})
        time.attrs = TFUT.time_counter.attrs
    else:
        time = TFUT.time_counter
    ds_FUT_T = xr.Dataset(coords={"time_counter": time, 
                                  "nav_lat": TFUT.nav_lat,"nav_lon": TFUT.nav_lon})
    if lent == 72:
        dummy = (("time_counter", "y", "x"), np.zeros((len(time), len(TFUT.y), len(TFUT.x))))
        ds_FUT_T["sozotwot"] = dummy
        ds_FUT_T["sozoiwot"] = dummy
        ds_FUT_T["sozoowot"] = dummy
        ds_FUT_T["sometwot"] = dummy
        ds_FUT_T["someiwot"] = dummy
        ds_FUT_T["someowot"] = dummy
        ds_FUT_T["sozotwot"][0:lent, :, :] = calc_work(UFUT.sozotaux, 
                                                 UFUT.vozocrtx.isel(depthu=0), uvt="u")
        ds_FUT_T["sozoiwot"][0:lent, :, :] = calc_work(UFUT.utau_iceoce * IFUT.siconc, 
                                                 UFUT.vozocrtx.isel(depthu=0), uvt="u")
        ds_FUT_T["sozoowot"][0:lent, :, :] = calc_work(UFUT.utau_atmoce * (1 - IFUT.siconc), 
                                                 UFUT.vozocrtx.isel(depthu=0), uvt="u")
        ds_FUT_T["sometwot"][0:lent, :, :] = calc_work(VFUT.sometauy, 
                                                 VFUT.vomecrty.isel(depthv=0), uvt="v")
        ds_FUT_T["someiwot"][0:lent, :, :] = calc_work(VFUT.vtau_iceoce * IFUT.siconc, 
                                                 VFUT.vomecrty.isel(depthv=0), uvt="v")
        ds_FUT_T["someowot"][0:lent, :, :] = calc_work(VFUT.vtau_atmoce * (1 - IFUT.siconc), 
                                                 VFUT.vomecrty.isel(depthv=0), uvt="v")
        ds_FUT_T["sozotwot"][lent, :, :] = ds_FUT_T["sozotwot"][lent-1, :, :].data
        ds_FUT_T["sozoiwot"][lent, :, :] = ds_FUT_T["sozoiwot"][lent-1, :, :].data
        ds_FUT_T["sozoowot"][lent, :, :] = ds_FUT_T["sozoowot"][lent-1, :, :].data
        ds_FUT_T["sometwot"][lent, :, :] = ds_FUT_T["sometwot"][lent-1, :, :].data
        ds_FUT_T["someiwot"][lent, :, :] = ds_FUT_T["someiwot"][lent-1, :, :].data
        ds_FUT_T["someowot"][lent, :, :] = ds_FUT_T["someowot"][lent-1, :, :].data
    else:    
        ds_FUT_T["sozotwot"] = (("time_counter", "y", "x"), calc_work(UFUT.sozotaux, 
                                                              UFUT.vozocrtx.isel(depthu=0), uvt="u"))
        ds_FUT_T["sozoiwot"] = (("time_counter", "y", "x"), calc_work(UFUT.utau_iceoce * IFUT.siconc, 
                                                              UFUT.vozocrtx.isel(depthu=0), uvt="u"))
        ds_FUT_T["sozoowot"] = (("time_counter", "y", "x"), calc_work(UFUT.utau_atmoce * (1 - IFUT.siconc), 
                                                              UFUT.vozocrtx.isel(depthu=0), uvt="u"))
        ds_FUT_T["sometwot"] = (("time_counter", "y", "x"), calc_work(VFUT.sometauy, 
                                                              VFUT.vomecrty.isel(depthv=0), uvt="v"))
        ds_FUT_T["someiwot"] = (("time_counter", "y", "x"), calc_work(VFUT.vtau_iceoce * IFUT.siconc, 
                                                              VFUT.vomecrty.isel(depthv=0), uvt="v"))
        ds_FUT_T["someowot"] = (("time_counter", "y", "x"), calc_work(VFUT.vtau_atmoce * (1 - IFUT.siconc), 
                                                              VFUT.vomecrty.isel(depthv=0), uvt="v"))
    ds_FUT_T.to_netcdf("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-windwork/"
                       + "CREG12.L75-FUT08_y" + str(y) + ".5d_windwork.nc")

### Compute climatological seasonal cycle

In [24]:
def pre(ds):
    try:
        ds = ds.drop("time_centered")
    except:
        ds = ds
    return ds

In [25]:
REFww = xr.open_mfdataset("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-windwork/"
                          + "CREG12.L75-REF08_y????.5d_windwork.nc", preprocess=pre)
FUTww = xr.open_mfdataset("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-windwork/"
                          + "CREG12.L75-FUT08_y????.5d_windwork.nc", preprocess=pre)

In [26]:
REFsc = REFww.groupby("time_counter.dayofyear").mean("time_counter")
FUTsc = FUTww.groupby("time_counter.dayofyear").mean("time_counter")

In [27]:
icemod_data_filesREF = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/2006/"
                                 + "CREG12.L75-REF08_*.5d_icemod.nc"))
IREF = xr.open_mfdataset(icemod_data_filesREF, parallel=True, preprocess=prepro)
tREF = xr.cftime_range(start="0001-01-03", periods=73, freq="5D", calendar="noleap")
timeREF = xr.DataArray(tREF, coords={"time_counter": tREF})
timeREF.attrs = IREF.time_counter.attrs

In [28]:
tFUT = xr.cftime_range(start="0001-01-03", periods=73, freq="5D", calendar="noleap")
timeFUT = xr.DataArray(tFUT, coords={"time_counter": tFUT})
timeFUT.attrs = IREF.time_counter.attrs

In [29]:
REFsc = REFsc.rename({"dayofyear": "time_counter"})
REFsc["time_counter"] = timeREF
FUTsc = FUTsc.rename({"dayofyear": "time_counter"})
FUTsc["time_counter"] = timeFUT

In [30]:
REFsc.to_netcdf("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-windwork/"
                + "CREG12.L75-REF08_clim.5d_windwork.nc")
FUTsc.to_netcdf("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-windwork/"
                + "CREG12.L75-FUT08_clim.5d_windwork.nc")